# **Data Cleaning Notebook**

## Objectives

* Check for duplicates, missing values
* Clean the data
* Split the data into train and test sets

## Inputs

* Kaggle data file - inputs/datasets/raw/house-prices/house-price/house_prices_records.csv
* Kaggle data file - inputs/datasets/raw/house-prices/house-price/inherited_house.csv

## Outputs

* outputs/datasets/cleaned/TestSetCleaned.csv
* outputs/datasets/cleaned/TrainSetCleaned.csv

---

# Change working directory

Accessing the current directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Making sure working in the child of the workspace directory

In [ ]:
os.chdir('/workspaces/milestone-project-heritage-housing-issues')
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Explore the Kaggle Data

* Load Kaggle Data

In [ ]:
import pandas as pd
allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']
df = pd.read_csv(
    f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/"
    f"house_prices_records.csv", na_values=allowed_nans, keep_default_na=False)
df.head()

* Change object type data to numerical data

In [ ]:
df['BsmtExposure'] = df['BsmtExposure'].replace(
    {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4})
df['BsmtFinType1'] = df['BsmtFinType1'].replace(
    {'None': 0, 'Unf': 1, 'LwQ': 2, 'BLQ': 3, 'Rec': 4, 'ALQ': 5, 'GLQ': 6})
df['GarageFinish'] = df['GarageFinish'].replace(
    {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3})
df['KitchenQual'] = df['KitchenQual'].replace(
    {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4})

* Check the distribution and shape of a variable with missing data

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

* Run Profile Report

In [ ]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

* Check for duplicated data

In [ ]:
df[df.duplicated()]

* Check for missing values

In [ ]:
df.isnull().sum()

---

## Deciding How to Handle Missing Data

### 2ndFlrSF 

* 86 missing values - 5.9% of the data
* Missing data could be becasue there is no second floor
* 53.5% of the data is 0
* I will take from this that the missing data also means there is no second floor and so replace the missing values with 0

In [ ]:
df['2ndFlrSF'] = df['2ndFlrSF'].fillna(0)

### BedroomAbvGr

* 99 values are missing - 6.8% of the data
* Only 6 entries, or 0.4% of the data, is 0 so it seems unlikely that the missing data would be due to there being no bedrooms above grade
* 51.3% of the data is 3, the mean of the data is 2.87 and the median is 3
* I will replace the missing data with the median value

In [ ]:
df['BedroomAbvGr'] = df['BedroomAbvGr'].fillna(3)

### BsmtFinType1

* There are 145 missing values - 9.9% of the data
* Some of these may be due to there not being a basement and there is a column that details if a basement is present I could use this to replace these NaN's with 0 which represents 'None'but, for simplicity, I will replace the other missing values with the mode value of 1, which represents 'unf'

In [ ]:
import numpy as np

df['BsmtFinType1'] = df['BsmtFinType1'].fillna(0)

### EnclosedPorch 

* There are 1324 values missing - 90.7% of the data
* Missing data could be because there is no enclosed porch. As there is no other column relating to an enclosed porch there is no way of testing this hypothesis
* There is no evidence of correlation between SalePrice and EnclosedPorch
* Due to the high volume of missing data I will drop this column

In [ ]:
df.drop(columns=['EnclosedPorch'], inplace=True)

### GarageFinish

* There are 162 missing values - 11.1% of the data
* Some of these these may be due to there not being a garage. I could replace these values with 'None' using the GarageArea column (using the logic there is no garage if the GarageArea is 0) but, for simplicity, I will replace the other missing values with the mode value, which is 1. This represents'Unf'

In [ ]:
df['GarageFinish'] = df['GarageFinish'].fillna(0)

### GarageYrBlt 

* There are 81 missing entries - 5.5% of the data
* I would expect there to be a correlation between the year the property was built and the year the garage was built as it would be common for these to be built at the same time

In [ ]:
df_year = df[['YearBuilt', 'GarageYrBlt']]

In [ ]:
df_year.corr(method='spearman')

In [ ]:
df_year.corr(method='pearson')

* There is a high correlation between GarageYrBly and YearBuilt. I will explore this further. I would expect there to be less correlation in the earlier years due to cars not having been invented or not being common. I will set the limit to 0.6 as this is when there stops being a strong correlation

In [ ]:
df_filtered_spearman = df_year[df_year['YearBuilt'] < 1988]
df_filtered_spearman.corr(method='spearman')

The spearman coefficient falls below 0.6 when the year is changed to be filtered at below 1987

In [ ]:
df_filtered_pearson = df_year[df_year['YearBuilt'] < 1995]
df_filtered_pearson.corr(method='pearson')

The pearson coefficient falls below 0.6 when the year is changed to be filtered at below 1994

* Some of these these may be due to there not being a garage. I will replace these values with 'None' using the GarageArea column (again using the logic there is no garage if the GarageArea is 0)

In [ ]:
df["GarageYrBlt"] = np.where(
    df["GarageArea"] == 0, df['GarageYrBlt'].fillna(0), None)

* A strong correlation between GarageYrBlt and YearBuilt is shown for those properties built after 1995 for Pearson and 1988 for Spearman
* For properties built after 1995 I will replace the missing GarageYrBlt values with the YearBuilt values
* For properties built prior to 1995 I will replace the missing values with the mean of GarageYrBlt - this is 1979

In [ ]:
df['GarageYrBlt'] = np.where(
    df['YearBuilt'] > 1995, df['GarageYrBlt'].fillna(
        df['YearBuilt']), df['GarageYrBlt'].fillna(1979))

### LotFrontage

* There are 259 missing values - 17.7% of the data
* I will replace missing values with the mean, which is 70

In [ ]:
df['LotFrontage'] = df['LotFrontage'].fillna(70)

### MasMasVnrArea

* There are 8 missing values - 0.5%
* 59% of values are 0 and 0 is also the median value so, I will replace missing values with 0

In [ ]:
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

### WoodDeckSF

* There are 1305 missing values - 89.4% of the data
* There is no evidence of correlation between SalePrice and WoodDeckSF
* Due to the volume of missing data I will drop this column

In [ ]:
df.drop(columns=['WoodDeckSF'], inplace=True)

In [ ]:
df.info()

* Rename columns with titles starting in a number due to errors running the code to check if the data type is int

In [ ]:
df = df.rename(columns={'1stFlrSF': 'FirFlrSF', '2ndFlrSF': 'SecFlrSF'})

* Check if any of the columns that are type float are actually integers

In [ ]:
df.MasVnrArea.apply(float.is_integer).all()
df.LotFrontage.apply(float.is_integer).all()
df.GarageYrBlt.apply(float.is_integer).all()
df.BedroomAbvGr.apply(float.is_integer).all()
df.SecFlrSF.apply(float.is_integer).all()
df.BsmtFinType1.apply(float.is_integer).all()
df.GarageFinish.apply(float.is_integer).all()

* They all are so, change all columns of type float, in original dataframe, to integers

In [ ]:
floats = [
    'MasVnrArea', 'LotFrontage', 'GarageYrBlt',
    'BedroomAbvGr', 'SecFlrSF', 'BsmtFinType1',
    'GarageFinish']
df[floats] = df[floats].astype(int)

In [ ]:
df.info()

---

# Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

---

# Push files to Repo

In [ ]:
import os
try:
    os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
    print(e)

## Train Set

In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

## Test Set

In [ ]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)